# Model fitting. Determine Teff and log(g).

In [17]:
%matplotlib qt

In [2]:
import sys
sys.path.append('/mnt/c/Users/luukv/Documenten/NatuurSterrkenkundeMasterProject/CodeMP/MasterProject')

import numpy as np
import matplotlib.pyplot as plt
import os
import re
from scipy.interpolate import CubicSpline
from PyAstronomy import pyasl

from import_data import BailerJones
from model_functions import *
from functions import import_spectra, extract_spectrum_within_range

Import models and spectrum (~21 seconds)

In [3]:
# Import models for given galaxy
models = import_models('Milkyway')
# Import spectrum of given object
spectra = import_spectra('4U1538_52')

# Name of object
object_name = '4U1538-52'
# Distance to object
distance = BailerJones().loc[BailerJones().id == object_name]['r_med_photogeo'].reset_index(drop=True).at[0]

In [4]:
filtered_models = models_in_interval(models, 30000, 35000, 3.0, 3.2)

### Perform $\chi^{2}$ fit for the $H\gamma$ (4340.26 $\AA$) for 4U1538-52
Crop the $H\gamma$ line

In [5]:
# Central wavelength of the line
line_wav = 4340.26 # Angstrom
# Range around the line
delta = 10
# Define a doppler shift
doppler = 2.3

Define the continuum and the spectral line

In [6]:
# Define the spectral line wavelenght range.
line_left = 4335 + doppler
line_right = 4341 + doppler

def extract_continuum(wavelengths: np.array, flux: np.array, start: float, end: float, line_left: float, line_right: float)->tuple:
    """
    Extract continuum from spectrum

    Args:
        wavelengths (np.array): List with wavelength
        flux (np.array): List with flux
        start (float): Start of spectrum
        end (float): End of spectrum
        line_left (float): Start spectral line
        line_right (float): End spectral line

    Returns:
        tuple: (Wavelength, Flux)
    """
    # Find indices corresponding to the specified wavelength range
    indices_left = np.where((wavelengths >= start) & (wavelengths <= line_left))[0]
    indices_right = np.where((wavelengths >= line_right) & (wavelengths <= end))[0]
    indices = np.concatenate((indices_left, indices_right))

    # Extract wavelength and flux values within the range
    extracted_wavelengths = wavelengths[indices]
    extracted_flux = flux[indices]

    return extracted_wavelengths, extracted_flux

# Get the spectrum (ADD A DOPPLERSHIFT)
wav = spectra[0][0] + doppler
flux = spectra[0][1]

# Get the continuum around the line
wav_cont, flux_cont = extract_continuum(wav, flux, line_wav-delta, line_wav+delta, line_left, line_right)
# Get the line
wav_line, flux_line = extract_spectrum_within_range(wav, flux, line_left, line_right)
# Get the whole range
wav_all, flux_all = extract_spectrum_within_range(wav, flux, line_wav-delta, line_wav+delta)

# Linear fit to continuum
p1 = np.poly1d(np.polyfit(wav_cont, flux_cont, 1))

# Plot
plt.plot(wav_all, flux_all)
for model in filtered_models.values():
    wav, flux = extract_spectrum_within_range(np.array(model['WAVELENGTH']), np.array(model['FLUX']), line_wav-delta, line_wav+delta)
    plt.plot(wav,flux)

plt.plot(wav_all, flux_all)
plt.plot(wav_cont, flux_cont)
plt.plot(wav_all, p1(wav_all), color='orange')
plt.show()

Normalize line with linear fit.

In [7]:
# Normalize line
flux_line_norm = flux_line / p1(wav_line)
plt.plot(wav_line, flux_line_norm)
for model in filtered_models.values():
    wav, flux = extract_spectrum_within_range(np.array(model['WAVELENGTH']), np.array(model['FLUX']), line_wav-delta, line_wav+delta)
    plt.plot(wav,flux)
plt.show()

Perform $\chi^{2}$ fit for all models.

In [8]:
# Signal to noise ratio
SNR = 9

def chi_squared(wav_model, flux_model, wav_line, flux_line, SNR):

    # Get the flux for every wavelenght of the data.
    # Create a CubicSpline object
    cubic_spline = CubicSpline(wav_model, flux_model)

    # Interpolate intensity at the desired wavelengths
    flux_model_inter = cubic_spline(wav_line)

    # Calculate chi-squared
    chi2 = 0
    for i in range(len(flux_line)):
        chi2 += ( (flux_model_inter[i] - flux_line[i]) / (1 / SNR) ) ** 2
    chi2 /= len(wav_line)

    return chi2

chi_squared(wav, flux, wav_line, flux_line_norm, SNR)

0.9844657956096715

In [9]:
chi2 = {}
for key, model in models.items():
    # Extract the line from the model
    wav, flux = extract_spectrum_within_range(np.array(model['WAVELENGTH']), np.array(model['FLUX']), line_wav-delta, line_wav+delta)

    # Calculate chi-squared for model
    chi2[key] = chi_squared(wav, flux, wav_line, flux_line_norm, SNR)

In [10]:
print(f"The minimum chi-squared: {chi2[min(chi2, key=chi2.get)]}")
print(f"This is for model: {min(chi2, key=chi2.get)}")

The minimum chi-squared: 0.7211039654629444
This is for model: T38000logg3.6


In [11]:
plt.plot(chi2.values())
plt.ylabel(r'$\chi^{2}$', size=20)
plt.show()

In [23]:
wav, flux = extract_spectrum_within_range(np.array(models['T38000logg3.6']['WAVELENGTH']), np.array(models['T38000logg3.6']['FLUX']), line_wav-delta, line_wav+delta)
plt.scatter(wav, flux, color='orange')
plt.plot(wav_line, flux_line_norm)
plt.show()

Apply dopplerbroadening

In [13]:
# PyAstronomy requires equal spacings between the wavelength intervals
# PyAstronomy provides a packege for this.
w2, f2 = pyasl.equidistantInterpolation(wav, flux, 0.01)

# Obtain the broadened spectrum using
# vsini = 13.3 km/s and no limb-darkening
rflux = pyasl.rotBroad(w2, f2, 0.0, 200)

plt.plot(w2, rflux, label='Broadened')
plt.plot(wav, flux, label='original')
plt.plot(w2, f2, label='interpolated')
plt.legend()
plt.show()

Calculate $\chi^{2}$ with doppler broadening applied.

In [25]:
chi2 = {}
for key, model in models.items():
    # Extract the line from the model
    wav, flux = extract_spectrum_within_range(np.array(model['WAVELENGTH']), np.array(model['FLUX']), line_wav-delta, line_wav+delta)

    # Apply doppler broadening
    w2, f2 = pyasl.equidistantInterpolation(wav, flux, "2x")

    # Obtain the broadened spectrum using
    rflux = pyasl.rotBroad(w2, f2, 0.0, 180)

    # Calculate chi-squared for model
    chi2[key] = chi_squared(w2, rflux, wav_line, flux_line_norm, SNR)

In [28]:
print(f"The minimum chi-squared: {chi2[min(chi2, key=chi2.get)]}")
print(f"This is for model: {min(chi2, key=chi2.get)}")

plt.plot(chi2.values())
plt.ylabel(r'$\chi^{2}$', size=20)
plt.show()

The minimum chi-squared: 0.4834778307644931
This is for model: T26000logg2.8


In [27]:
wav, flux = extract_spectrum_within_range(np.array(models['T26000logg2.8']['WAVELENGTH']), np.array(models['T26000logg2.8']['FLUX']), line_wav-delta, line_wav+delta)

# Apply doppler broadening
w2, f2 = pyasl.equidistantInterpolation(wav, flux, "2x")

# Obtain the broadened spectrum using
rflux = pyasl.rotBroad(w2, f2, 0.0, 180)

plt.plot(wav_line, flux_line_norm, color='blue')
plt.plot(w2, rflux, color='orange')
plt.show()